In [1]:
from __future__ import print_function
import os
import neat

import pandas as pd
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim


from explaneat.core.backprop import NeatNet
from explaneat.core import backprop
from explaneat.core.backproppop import BackpropPopulation
# from explaneat.visualization import visualize
from explaneat.core.experiment import ExperimentReporter
from explaneat.core.utility import one_hot_encode
from explaneat.core.utility import MethodTimer


from sklearn import datasets, metrics
from sklearn.preprocessing import StandardScaler, normalize, OneHotEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

from copy import deepcopy

import time
from datetime import datetime

from explaneat.core.neuralneat import NeuralNeat as nneat

In [2]:
RANDOM_SEED      = 42
torch.manual_seed(RANDOM_SEED)

USE_CUDA = torch.cuda.is_available()
USE_CUDA = False
device = torch.device("cuda:1" if USE_CUDA else "cpu")



In [3]:
torch.cuda.is_available()

False

In [4]:

def xor(a, b, threshold = 0.5):
    response = False
    if a > threshold and b < threshold:
        response = True
    if a < threshold and b > threshold:
        response = True
    # return (1.0, 0.0) if response else (0.0, 1.0)
    return 1.0 if response else 0.0
    

def create_n_points(n, size, min=0.0, max=1.0):
    data = []
    for _ in range(n):
        data.append([
            random.uniform(min, max) for ii in range(size)
        ])

    return data

# correct solution:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference

def overUnder(val, threshold):
    return 1. if val > threshold else 0

xor_inputs = create_n_points(400, 2, -1, 1)

xor_outputs = [
    [xor(tup[0], tup[1], 0)] for tup in xor_inputs
]

In [5]:
xor_inputs[:5]

[[0.870015669339151, 0.04251848380257339],
 [-0.09132695985153005, -0.06454170656147773],
 [0.14681228380560185, 0.2647219775028271],
 [0.46285542547505343, -0.8571944154754261],
 [0.5377812869629481, -0.3478608864242061]]

In [6]:
xor_outputs[:5]

[[0.0], [0.0], [0.0], [1.0], [1.0]]

# Set up run


In [7]:
config_path = "./config_xor"
base_config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)

In [8]:
def instantiate_population(config, xs, ys, saveLocation):

    if not os.path.exists(saveLocation):
        os.makedirs(saveLocation)
        
    config.save(os.path.join(saveLocation, 'config.conf'))

    # Create the population, which is the top-level object for a NEAT run.
    with MethodTimer("Backprop everything"):
        p = BackpropPopulation(config, 
                                xs, 
                                ys, 
                                criterion=nn.BCEWithLogitsLoss())

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5, filename_prefix=str(saveLocation) + "checkpoint-" ))
    bpReporter = backprop.BackpropReporter(True)
    p.add_reporter(bpReporter)
    p.add_reporter(ExperimentReporter(saveLocation))
    
    return p

In [9]:
def eval_genomes(genomes, config):
    
    print(genomes)
    loss = nn.BCELoss()
    loss = loss.to(device)
    for genome_id, genome in genomes.items():
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        preds = []
        for xi in xor_inputs:
            preds.append(net.activate(xi))
        genome.fitness = float(1./loss(torch.tensor(preds).to(device), torch.tensor(xor_outputs)))

In [10]:
config = base_config
saveLocation = './'
maxNGenerations = 3
p = instantiate_population(config, xor_inputs, xor_outputs, saveLocation)
# Run for up to nGenerations generations.
winner = p.run(eval_genomes, maxNGenerations, nEpochs = 5)

g = p.best_genome


The function - Backprop everything - has just started at 1632343889.854615
The function - Backprop everything - took 0.044628143310546875 seconds to complete
The function - generationStart - has just started at 1632343889.899742

 ****** Running generation 0 ****** 

The function - generationStart - took 0.00016427040100097656 seconds to complete
The function - pre_backprop - has just started at 1632343889.899939
The function - pre_backprop - took 9.202957153320312e-05 seconds to complete
The function - backprop - has just started at 1632343889.9001012
about to start backprop with 5 epochs
mean improvement: -0.0012103368198654208
best improvement: -0.0016138206778952835
best loss: 0.5130461264252956
The function - backprop - took 0.08482789993286133 seconds to complete
The function - post_backprop - has just started at 1632343889.984957
The function - post_backprop - took 2.288818359375e-05 seconds to complete
The function - evaluate fitness - has just started at 1632343889.985062
{1: 

In [11]:
print(g)

Key: 6
Fitness: 1.0407109260559082
Nodes:
	0 DefaultNodeGene(key=0, bias=0.05669275103529431, response=1.0, activation=sigmoid, aggregation=sum)
	1 DefaultNodeGene(key=1, bias=2.283953286287462, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-2, 0), weight=1.0906448201827157, enabled=False)
	DefaultConnectionGene(key=(-2, 1), weight=1.8134750207166896, enabled=True)
	DefaultConnectionGene(key=(-1, 0), weight=0.6403394255827368, enabled=True)


In [12]:
h = deepcopy(g)

In [13]:
print(h)

Key: 6
Fitness: 1.0407109260559082
Nodes:
	0 DefaultNodeGene(key=0, bias=0.05669275103529431, response=1.0, activation=sigmoid, aggregation=sum)
	1 DefaultNodeGene(key=1, bias=2.283953286287462, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-2, 0), weight=1.0906448201827157, enabled=False)
	DefaultConnectionGene(key=(-2, 1), weight=1.8134750207166896, enabled=True)
	DefaultConnectionGene(key=(-1, 0), weight=0.6403394255827368, enabled=True)


In [14]:
h.mutate_add_node(p.config.genome_config)
h.mutate_add_node(p.config.genome_config)
h.mutate_add_node(p.config.genome_config)
h.mutate_add_node(p.config.genome_config)
h.mutate_add_connection(p.config.genome_config)
h.mutate_add_connection(p.config.genome_config)
h.mutate_add_connection(p.config.genome_config)
h.mutate_add_connection(p.config.genome_config)
h.mutate_add_connection(p.config.genome_config)
h.mutate_add_connection(p.config.genome_config)
h.mutate_add_connection(p.config.genome_config)

In [15]:
print(h)

Key: 6
Fitness: 1.0407109260559082
Nodes:
	0 DefaultNodeGene(key=0, bias=0.05669275103529431, response=1.0, activation=sigmoid, aggregation=sum)
	1 DefaultNodeGene(key=1, bias=2.283953286287462, response=1.0, activation=sigmoid, aggregation=sum)
	3 DefaultNodeGene(key=3, bias=-1.242454911423579, response=1.0, activation=sigmoid, aggregation=sum)
	4 DefaultNodeGene(key=4, bias=-0.601721113606463, response=1.0, activation=sigmoid, aggregation=sum)
	5 DefaultNodeGene(key=5, bias=0.5964337650762562, response=1.0, activation=sigmoid, aggregation=sum)
	6 DefaultNodeGene(key=6, bias=2.176185665300897, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-2, 0), weight=1.0906448201827157, enabled=False)
	DefaultConnectionGene(key=(-2, 1), weight=1.8134750207166896, enabled=False)
	DefaultConnectionGene(key=(-2, 3), weight=1.0, enabled=True)
	DefaultConnectionGene(key=(-2, 4), weight=1.0, enabled=True)
	DefaultConnectionGene(key=(-2, 5), weight=1.0, enable

In [16]:
# h.add_connection(p.config.genome_config, 16, 17, 0.2, True)
# h.add_connection(p.config.genome_config, 17, 0, 0.2, True)

In [17]:
print(h)

Key: 6
Fitness: 1.0407109260559082
Nodes:
	0 DefaultNodeGene(key=0, bias=0.05669275103529431, response=1.0, activation=sigmoid, aggregation=sum)
	1 DefaultNodeGene(key=1, bias=2.283953286287462, response=1.0, activation=sigmoid, aggregation=sum)
	3 DefaultNodeGene(key=3, bias=-1.242454911423579, response=1.0, activation=sigmoid, aggregation=sum)
	4 DefaultNodeGene(key=4, bias=-0.601721113606463, response=1.0, activation=sigmoid, aggregation=sum)
	5 DefaultNodeGene(key=5, bias=0.5964337650762562, response=1.0, activation=sigmoid, aggregation=sum)
	6 DefaultNodeGene(key=6, bias=2.176185665300897, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-2, 0), weight=1.0906448201827157, enabled=False)
	DefaultConnectionGene(key=(-2, 1), weight=1.8134750207166896, enabled=False)
	DefaultConnectionGene(key=(-2, 3), weight=1.0, enabled=True)
	DefaultConnectionGene(key=(-2, 4), weight=1.0, enabled=True)
	DefaultConnectionGene(key=(-2, 5), weight=1.0, enable

In [18]:
p.config.genome_config.input_keys

[-1, -2]

In [19]:
h.nodes

{0: <neat.genes.DefaultNodeGene at 0x7fa7c882abe0>,
 1: <neat.genes.DefaultNodeGene at 0x7fa7c882aca0>,
 3: <neat.genes.DefaultNodeGene at 0x7fa7c8830430>,
 4: <neat.genes.DefaultNodeGene at 0x7fa7c8830d00>,
 5: <neat.genes.DefaultNodeGene at 0x7fa7c8830e20>,
 6: <neat.genes.DefaultNodeGene at 0x7fa7c8830f40>}

In [20]:
node_tracker = {node_id:{'depth':0, 'output_ids':[], 'input_ids':[]} for node_id in h.nodes}
for node_id in p.config.genome_config.input_keys:
    node_tracker[node_id] = {'depth':0, 'output_ids':[], 'input_ids':[]}
trace_stack = [node_id for node_id in p.config.genome_config.input_keys]

In [21]:
trace_stack

[-1, -2]

In [22]:
node_tracker

{0: {'depth': 0, 'output_ids': [], 'input_ids': []},
 1: {'depth': 0, 'output_ids': [], 'input_ids': []},
 3: {'depth': 0, 'output_ids': [], 'input_ids': []},
 4: {'depth': 0, 'output_ids': [], 'input_ids': []},
 5: {'depth': 0, 'output_ids': [], 'input_ids': []},
 6: {'depth': 0, 'output_ids': [], 'input_ids': []},
 -1: {'depth': 0, 'output_ids': [], 'input_ids': []},
 -2: {'depth': 0, 'output_ids': [], 'input_ids': []}}

In [23]:
for connection in h.connections:
    print(connection)
    node_tracker[connection[0]]['output_ids'].append(connection[1])
    node_tracker[connection[1]]['input_ids'].append(connection[0])

(-1, 0)
(-2, 0)
(-2, 1)
(-2, 3)
(3, 1)
(-2, 4)
(4, 0)
(-2, 5)
(5, 1)
(-2, 6)
(6, 5)
(5, 3)
(0, 5)
(0, 3)
(4, 6)
(6, 3)


In [24]:
while len(trace_stack) > 0:
    trace = trace_stack[0]
    my_depth = node_tracker[trace]['depth']
    next_depth = my_depth + 1
    for output_id in node_tracker[trace]['output_ids']:
        node_tracker[output_id]['depth'] = max(node_tracker[output_id]['depth'], next_depth)
        trace_stack.append(output_id)
    del(trace_stack[0])

In [25]:
node_tracker

{0: {'depth': 2, 'output_ids': [5, 3], 'input_ids': [-1, -2, 4]},
 1: {'depth': 5, 'output_ids': [], 'input_ids': [-2, 3, 5]},
 3: {'depth': 4, 'output_ids': [1], 'input_ids': [-2, 5, 0, 6]},
 4: {'depth': 1, 'output_ids': [0, 6], 'input_ids': [-2]},
 5: {'depth': 3, 'output_ids': [1, 3], 'input_ids': [-2, 6, 0]},
 6: {'depth': 2, 'output_ids': [5, 3], 'input_ids': [-2, 4]},
 -1: {'depth': 0, 'output_ids': [0], 'input_ids': []},
 -2: {'depth': 0, 'output_ids': [0, 1, 3, 4, 5, 6], 'input_ids': []}}

In [26]:
for node_id, node in node_tracker.items():
    node['output_layers']=[]
    node['needs_skip'] = False
    node['id'] = node_id
    for output_id in node['output_ids']:
        node['output_layers'].append(node_tracker[output_id]['depth'])
        if node_tracker[output_id]['depth'] > (node['depth']+1):
            node['needs_skip'] = True

Create input layer definitions

In [27]:
for node_id, node in node_tracker.items():
    node['input_layers'] = []
    node['skip_layer_input'] = False
    for input_id in node['input_ids']:
        node['input_layers'].append(node_tracker[input_id]['depth'])
        if node_tracker[input_id]['depth'] < (node['depth']-1):
            node['skip_layer_input'] = True

In [28]:
node_tracker

{0: {'depth': 2,
  'output_ids': [5, 3],
  'input_ids': [-1, -2, 4],
  'output_layers': [3, 4],
  'needs_skip': True,
  'id': 0,
  'input_layers': [0, 0, 1],
  'skip_layer_input': True},
 1: {'depth': 5,
  'output_ids': [],
  'input_ids': [-2, 3, 5],
  'output_layers': [],
  'needs_skip': False,
  'id': 1,
  'input_layers': [0, 4, 3],
  'skip_layer_input': True},
 3: {'depth': 4,
  'output_ids': [1],
  'input_ids': [-2, 5, 0, 6],
  'output_layers': [5],
  'needs_skip': False,
  'id': 3,
  'input_layers': [0, 3, 2, 2],
  'skip_layer_input': True},
 4: {'depth': 1,
  'output_ids': [0, 6],
  'input_ids': [-2],
  'output_layers': [2, 2],
  'needs_skip': False,
  'id': 4,
  'input_layers': [0],
  'skip_layer_input': False},
 5: {'depth': 3,
  'output_ids': [1, 3],
  'input_ids': [-2, 6, 0],
  'output_layers': [5, 4],
  'needs_skip': True,
  'id': 5,
  'input_layers': [0, 2, 2],
  'skip_layer_input': True},
 6: {'depth': 2,
  'output_ids': [5, 3],
  'input_ids': [-2, 4],
  'output_layers': [

In [29]:
layers = {}
for node_id, node in node_tracker.items():
    if not node['depth'] in layers:
        layers[node['depth']] = {
            'nodes':{node_id:node}
        }
    else:
        layers[node['depth']]['nodes'][node_id] = node
        
# Ensure all nodes have a layer index
for layer_id, layer in layers.items():
    layer_index = 0
    for node_id, node in layer['nodes'].items():
        node['layer_index'] = layer_index
        layer_index += 1
        
        
layers

{2: {'nodes': {0: {'depth': 2,
    'output_ids': [5, 3],
    'input_ids': [-1, -2, 4],
    'output_layers': [3, 4],
    'needs_skip': True,
    'id': 0,
    'input_layers': [0, 0, 1],
    'skip_layer_input': True,
    'layer_index': 0},
   6: {'depth': 2,
    'output_ids': [5, 3],
    'input_ids': [-2, 4],
    'output_layers': [3, 4],
    'needs_skip': True,
    'id': 6,
    'input_layers': [0, 1],
    'skip_layer_input': True,
    'layer_index': 1}}},
 5: {'nodes': {1: {'depth': 5,
    'output_ids': [],
    'input_ids': [-2, 3, 5],
    'output_layers': [],
    'needs_skip': False,
    'id': 1,
    'input_layers': [0, 4, 3],
    'skip_layer_input': True,
    'layer_index': 0}}},
 4: {'nodes': {3: {'depth': 4,
    'output_ids': [1],
    'input_ids': [-2, 5, 0, 6],
    'output_layers': [5],
    'needs_skip': False,
    'id': 3,
    'input_layers': [0, 3, 2, 2],
    'skip_layer_input': True,
    'layer_index': 0}}},
 1: {'nodes': {4: {'depth': 1,
    'output_ids': [0, 6],
    'input_ids':

In [30]:

LAYER_TYPE_CONNECTED = "CONNECTED"
LAYER_TYPE_INPUT = "INPUT"
LAYER_TYPE_OUTPUT = "OUTPUT"
for layer_id, layer in layers.items():
    layer['is_output_layer'] = False
    layer['is_input_layer'] = False
    layer['layer_type'] = LAYER_TYPE_CONNECTED
    # If I have the output node in me, then I am an output
    if 0 in layer['nodes']:
        layer['is_output_layer'] = True
        layer['layer_type'] = LAYER_TYPE_OUTPUT

    # If I have the first input in me, then I am the input
    if -1 in layer['nodes']:
        layer['is_input_layer'] = True
        layer['layer_type'] = LAYER_TYPE_INPUT
    biases = []

    layer['input_layers'] = []
    ## Compute the shape of required inputs
    for node_id, node in layer['nodes'].items():
        for in_layer in node['input_layers']:
            if in_layer not in layer['input_layers']:
                layer['input_layers'].append(in_layer)
    layer['input_layers'].sort()
    layer['input_shape'] = sum(len(layers[jj]['nodes']) for jj in layer['input_layers'])
    layer['weights_shape'] = (layer['input_shape'], len(layer['nodes']))


    # Handle output layer "edge" case
    if layer['is_output_layer']:
        layer['out_weights'] = []
        layer['bias'] = [h.nodes[node_id].bias for node_id, node in layer['nodes'].items()]
        layer['in_weights'] = [[0 for __ in layers[layer_id-1]['nodes']] for _ in layer['nodes']]
    # Handle input layer "edge" case
    elif layer['is_input_layer']:
        layer['in_weights'] = []
        layer['bias'] = []
        layer['out_weights'] = [[0 for __ in layers[layer_id+1]['nodes']] for _ in layer['nodes']]
    # Handle generic case
    else:
        layer['out_weights'] = [[0 for __ in layers[layer_id+1]['nodes']] for _ in layer['nodes']]
        layer['in_weights'] = [[0 for __ in layers[layer_id-1]['nodes']] for _ in layer['nodes']]

        layer['bias'] = [h.nodes[node_id].bias for node_id, node in layer['nodes'].items()]
        # else:
            # layer['bias'] = [0 for _ in layer['nodes']]
    
    layer_index = 0          
    for node_id, node in layer['nodes'].items():
        node['layer_index'] = layer_index
        layer_index += 1

#################################################
#################################################
#################################################
        ### POSSIBLY NEED THIS
        # if node['id'] < 0:
        #     biases.append(1)
        # else:
        #     biases.append(h.nodes[node['id']].bias)
        # for output_id in node['output_ids']:
        #     if (node['id'], output_id) in h.connections:
        #         if output_id in layers[layer_id + 1]['nodes']:
        #             layer['out_weights'][node['layer_index']][layers[layer_id + 1]['nodes'][output_id]['layer_index']] = h.connections[(node['id'], output_id)].weight
        #         else:
        #             # TODO: this is a skip layer to deal with
        #             pass
        #     else:
        #         # this is not a connection that exists, so default of 0 holds
        #         pass
        # for input_id in node['input_ids']:
        #     if (input_id, node['id']) in h.connections:
        #         if input_id in layers[layer_id - 1]['nodes']:
        #             layer['in_weights'][node['layer_index']][layers[layer_id -1]['nodes'][input_id]['layer_index']] = h.connections[(input_id, node['id'])].weight
        #         else:
        #             # TODO: this is a skip layer to deal with
        #             pass
        #     else:
        #         # this is not a connection that exists, so default of 0 holds
        #         pass
        #################################################
            ### END OF POSSIBLY NEEDING THIS
    #################################################
    #################################################
    #################################################
    #################################################


    # layer['out_tensor'] = torch.tensor(layer['out_weights'])
    # layer['bias'] = torch.tensor(layer['bias'])
    # layer['out_tensor_shape'] = layer['out_tensor'].shape
    # layer['in_tensor'] = torch.tensor(layer['in_weights'])
    # layer['in_tensor_shape'] = layer['in_tensor'].shape



    # Set up current weights
    layer['input_weights'] = np.zeros(layer['weights_shape'])
    layer_offset = 0
    # Check every layer and every node for connections
    print("my layer id is %s" % (layer_id))
    for input_layer_id in layer['input_layers']:
        input_layer = layers[input_layer_id]
        for node_id, node in input_layer['nodes'].items():
            for node_output_id in node['output_ids']:
                if node_output_id in layer['nodes']:
                    node_output = layer['nodes'][node_output_id]
                    # I HAVE THIS NODE!
                    # What is it's weight?
                    connection = h.connections[(node_id, node_output_id)]
                    print(connection)

                    if not connection.enabled:
                        continue
                    connection_weight = connection.weight
                    print(connection_weight)

                    in_weight_location = layer_offset + node['layer_index']
                    out_weight_location = node_output['layer_index']
                    print('location is')
                    print((out_weight_location, in_weight_location))
                    layer['input_weights'][in_weight_location][out_weight_location] = connection_weight
        layer_offset += len(input_layer['nodes'])

my layer id is 2
DefaultConnectionGene(key=(-1, 0), weight=0.6403394255827368, enabled=True)
0.6403394255827368
location is
(0, 0)
DefaultConnectionGene(key=(-2, 0), weight=1.0906448201827157, enabled=False)
DefaultConnectionGene(key=(-2, 6), weight=1.0, enabled=True)
1.0
location is
(1, 1)
DefaultConnectionGene(key=(4, 0), weight=1.0906448201827157, enabled=True)
1.0906448201827157
location is
(0, 2)
DefaultConnectionGene(key=(4, 6), weight=-0.899410082400078, enabled=True)
-0.899410082400078
location is
(1, 2)


KeyError: 6

In [31]:
layers

{5: {'nodes': {0: {'depth': 5,
    'output_ids': [],
    'input_ids': [-2, 2, 4],
    'output_layers': [],
    'needs_skip': False,
    'id': 0,
    'input_layers': [0, 4, 1],
    'skip_layer_input': True,
    'layer_index': 0}},
  'is_output_layer': True,
  'is_input_layer': False,
  'layer_type': 'OUTPUT',
  'input_layers': [0, 1, 4],
  'input_shape': 4,
  'weights_shape': (4, 1),
  'out_weights': [],
  'bias': [-0.32540032267570496],
  'in_weights': [[0]],
  'input_weights': array([[0.        ],
         [0.        ],
         [0.14471313],
         [0.14471313]])},
 4: {'nodes': {2: {'depth': 4,
    'output_ids': [0],
    'input_ids': [-2, 3, -1],
    'output_layers': [5],
    'needs_skip': False,
    'id': 2,
    'input_layers': [0, 3, 0],
    'skip_layer_input': True,
    'layer_index': 0}},
  'is_output_layer': False,
  'is_input_layer': False,
  'layer_type': 'CONNECTED',
  'input_layers': [0, 3],
  'input_shape': 3,
  'weights_shape': (3, 1),
  'out_weights': [[0]],
  'in_weig

Add in computation of skip layers

In [32]:
layers[2]

{'nodes': {5: {'depth': 2,
   'output_ids': [3],
   'input_ids': [-2, 4],
   'output_layers': [3],
   'needs_skip': False,
   'id': 5,
   'input_layers': [0, 1],
   'skip_layer_input': True,
   'layer_index': 0}},
 'is_output_layer': False,
 'is_input_layer': False,
 'layer_type': 'CONNECTED',
 'input_layers': [0, 1],
 'input_shape': 3,
 'weights_shape': (3, 1),
 'out_weights': [[0]],
 'in_weights': [[0]],
 'bias': [0.28752704139344826],
 'input_weights': array([[ 0.        ],
        [ 1.        ],
        [-1.04658333]])}

In [33]:
layers

{5: {'nodes': {0: {'depth': 5,
    'output_ids': [],
    'input_ids': [-2, 2, 4],
    'output_layers': [],
    'needs_skip': False,
    'id': 0,
    'input_layers': [0, 4, 1],
    'skip_layer_input': True,
    'layer_index': 0}},
  'is_output_layer': True,
  'is_input_layer': False,
  'layer_type': 'OUTPUT',
  'input_layers': [0, 1, 4],
  'input_shape': 4,
  'weights_shape': (4, 1),
  'out_weights': [],
  'bias': [-0.32540032267570496],
  'in_weights': [[0]],
  'input_weights': array([[0.        ],
         [0.        ],
         [0.14471313],
         [0.14471313]])},
 4: {'nodes': {2: {'depth': 4,
    'output_ids': [0],
    'input_ids': [-2, 3, -1],
    'output_layers': [5],
    'needs_skip': False,
    'id': 2,
    'input_layers': [0, 3, 0],
    'skip_layer_input': True,
    'layer_index': 0}},
  'is_output_layer': False,
  'is_input_layer': False,
  'layer_type': 'CONNECTED',
  'input_layers': [0, 3],
  'input_shape': 3,
  'weights_shape': (3, 1),
  'out_weights': [[0]],
  'in_weig

The question is, how do you code the operations to dynamically create the
NN? You need to be able to do the right things in the right order. You have
a list of possible operations:
* Matrix multiple
* Concatenate (for skip layers)
* Matrix addition (for biases)
* Output

The order of operations in each layer is:

1. Concatenate input tensors
2. Matrix multiply input tensors with weights
3. Add bias tensors
4. Apply activation function

I.e., `output = f(cat(inputs) + bias)` for a given activation function f

These computations can be computed back-to-front, and then executed forward

In [34]:
ACTIVATE_OPERATION = "ACTIVATE"
OUTPUT_OPERATION = "OUTPUT"
TENADD_OPERATION = "TENADD" # Tensor ADD
ADD_BIAS_OPERATION = "BIASADD"
TENMUL_OPERATION = "TENMUL"
TENCAT_OPERATION = "TENCAT"
order_of_operations = []

# for layer_id, layer in layers.items():
#     print(layer)
#     # Output for final layer
#     if layer['is_output_layer']:
#         order_of_operations.append(OUTPUT_OPERATION)
#     # Activate
#     order_of_operations.append(ACTIVATE_OPERATION)
#     # Add Bias
#     order_of_operations.append(ADD_BIAS_OPERATION)
#     # Matrix Multiply weights

    


#     print(order_of_operations)

#     break

for layer_id in range(len(layers)):
    layer = layers[layer_id]
    print(layer)

{'nodes': {-1: {'depth': 0, 'output_ids': [2], 'input_ids': [], 'output_layers': [4], 'needs_skip': True, 'id': -1, 'input_layers': [], 'skip_layer_input': False, 'layer_index': 0}, -2: {'depth': 0, 'output_ids': [0, 2, 3, 4, 5], 'input_ids': [], 'output_layers': [5, 4, 3, 1, 2], 'needs_skip': True, 'id': -2, 'input_layers': [], 'skip_layer_input': False, 'layer_index': 1}}, 'is_output_layer': False, 'is_input_layer': True, 'layer_type': 'INPUT', 'input_layers': [], 'input_shape': 0, 'weights_shape': (0, 2), 'in_weights': [], 'bias': [], 'out_weights': [[0], [0]], 'input_weights': array([], shape=(0, 2), dtype=float64)}
{'nodes': {4: {'depth': 1, 'output_ids': [0, 5], 'input_ids': [-2], 'output_layers': [5, 2], 'needs_skip': True, 'id': 4, 'input_layers': [0], 'skip_layer_input': False, 'layer_index': 0}}, 'is_output_layer': False, 'is_input_layer': False, 'layer_type': 'CONNECTED', 'input_layers': [0], 'input_shape': 2, 'weights_shape': (2, 1), 'out_weights': [[0]], 'in_weights': [[0,

In [35]:
a = torch.randn(2,3)
b = torch.randn(3, 1)
c = torch.matmul(a, b)

In [36]:
c

tensor([[-0.0890],
        [ 1.7597]])

In [37]:
layers

{5: {'nodes': {0: {'depth': 5,
    'output_ids': [],
    'input_ids': [-2, 2, 4],
    'output_layers': [],
    'needs_skip': False,
    'id': 0,
    'input_layers': [0, 4, 1],
    'skip_layer_input': True,
    'layer_index': 0}},
  'is_output_layer': True,
  'is_input_layer': False,
  'layer_type': 'OUTPUT',
  'input_layers': [0, 1, 4],
  'input_shape': 4,
  'weights_shape': (4, 1),
  'out_weights': [],
  'bias': [-0.32540032267570496],
  'in_weights': [[0]],
  'input_weights': array([[0.        ],
         [0.        ],
         [0.14471313],
         [0.14471313]])},
 4: {'nodes': {2: {'depth': 4,
    'output_ids': [0],
    'input_ids': [-2, 3, -1],
    'output_layers': [5],
    'needs_skip': False,
    'id': 2,
    'input_layers': [0, 3, 0],
    'skip_layer_input': True,
    'layer_index': 0}},
  'is_output_layer': False,
  'is_input_layer': False,
  'layer_type': 'CONNECTED',
  'input_layers': [0, 3],
  'input_shape': 3,
  'weights_shape': (3, 1),
  'out_weights': [[0]],
  'in_weig

In [38]:
x_input = nn.Parameter(torch.tensor([0.3, 0.4], dtype=torch.float64))

In [39]:
class Net(nn.Module):
    def __init__(self, layers):
        super(Net, self).__init__()  # just run the init of parent class (nn.Module)
        self.weights = {layer_id: self._tt(layer['input_weights'].copy()) for layer_id, layer in layers.items()}
        self.biases = {layer_id: self._tt(layer['bias'].copy()) for layer_id, layer in layers.items()}
        self.layer_types = {layer_id: layer['layer_type'] for layer_id, layer in layers.items()}
        self.layer_inputs = {layer_id: layer['input_layers'] for layer_id, layer in layers.items()}
        self.n_layers = len(layers)

        self._outputs = None

        for w_id, w in self.weights.items():
            self.register_parameter(name ="weight_%s"%w_id, param=w)

        for b_id, b in self.biases.items():
            self.register_parameter(name = "bias_%s"%b_id, param=b)

        # x = F.relu(self.fc1(x))

        # self.fc2 = nn.Linear(512, 10)

    @staticmethod
    def _tt(mat):
        return torch.nn.Parameter(torch.tensor(mat,dtype=torch.float64), requires_grad=True)
    def forward(self, x):
        # print("running forward")
        self._outputs = {}
        for layer_id in range(self.n_layers):
            layer_input = None
            layer_type = self.layer_types[layer_id]

            # print(layer_id)
            # print(layer_type)

            if layer_type == LAYER_TYPE_INPUT:
                self._outputs[layer_id] = x
                continue
            if layer_type == LAYER_TYPE_CONNECTED:
                layer_input = torch.cat([self._outputs[ii] for ii in self.layer_inputs[layer_id]])
            if layer_type == LAYER_TYPE_OUTPUT:
                layer_input = torch.cat([self._outputs[ii] for ii in self.layer_inputs[layer_id]])

            # print(layer_input)
            # print(self.weights[layer_id])
            # print(self.biases[layer_id])

            self._outputs[layer_id] = torch.sigmoid( torch.matmul(layer_input, self.weights[layer_id]) + self.biases[layer_id] )

            if layer_type == LAYER_TYPE_OUTPUT:
                return self._outputs[layer_id]


In [40]:
net = Net(layers)

In [41]:
net.biases

{5: Parameter containing:
 tensor([-0.3254], dtype=torch.float64, requires_grad=True),
 4: Parameter containing:
 tensor([-0.9947], dtype=torch.float64, requires_grad=True),
 3: Parameter containing:
 tensor([1.3604], dtype=torch.float64, requires_grad=True),
 1: Parameter containing:
 tensor([-0.7886], dtype=torch.float64, requires_grad=True),
 2: Parameter containing:
 tensor([0.2875], dtype=torch.float64, requires_grad=True),
 0: Parameter containing:
 tensor([], dtype=torch.float64, requires_grad=True)}

In [42]:
net._outputs

In [43]:
result = net.forward(x_input)
result

tensor([0.4499], dtype=torch.float64, grad_fn=<SigmoidBackward>)

In [44]:
expected_result = torch.tensor(xor(x_input[0], x_input[1]))
expected_result

tensor(0.)

In [45]:
learning_rate = 0.1
# x_input = torch.randn(1, 1, 2, dtype=torch.float64)
with MethodTimer('benchmark'):

    for k in range(25):
        for n in range(400):
            x_input = nn.Parameter(torch.tensor([np.random.rand(), np.random.rand()], dtype=torch.float64))


            result = net.forward(x_input)
            expected_result = torch.tensor(xor(x_input[0], x_input[1]), dtype=torch.float64)
            # print(x_input)
            # print(expected_result)
            # print(result)
            loss_fn = torch.nn.MSELoss(reduction='sum')
            loss = loss_fn(result, expected_result)
            net.zero_grad()
            loss.backward()
            with torch.no_grad():
                # for parameter in net.parameters():
                    # parameter.data -= learning_rate*parameter.grad.data
                    # weight -= learning_rate * weight.grad
                for  w_id, weight in net.weights.items():
                    try:
                        weight -= learning_rate * weight.grad
                    except TypeError:
                        continue
                for  b_id, bias in net.biases.items():
                    try:
                        bias -= learning_rate * bias.grad
                    except TypeError:
                        continue





The function - benchmark - has just started at 1628331614.001492
The function - benchmark - took 5.811469078063965 seconds to complete


In [46]:
# net(torch.tensor(xor_inputs, dtype=torch.float64))

In [47]:
net.biases[1].grad

tensor([0.0192], dtype=torch.float64)

In [48]:
result

tensor([0.5588], dtype=torch.float64, grad_fn=<SigmoidBackward>)

In [49]:
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)


In [50]:
optimizer.zero_grad()
output = net(x_input)
loss = loss_fn(result, expected_result)


In [51]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.0763],
        [-0.0542],
        [ 0.3221],
        [ 0.2249]], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([[-0.3051],
        [-0.1099],
        [ 0.7828]], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([[-0.0032],
        [-0.0108],
        [ 0.9855]], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([[-0.3092],
        [ 0.8493]], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([[ 0.0013],
        [ 0.9976],
        [-1.0488]], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([], size=(0, 2), dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([0.0632], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([-1.2436], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([1.3353], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([-1.0654], dtype=torch.float64, requir

In [52]:
myW = net.weights[1]

In [53]:
myW

Parameter containing:
tensor([[-0.3092],
        [ 0.8493]], dtype=torch.float64, requires_grad=True)

In [54]:
loss

tensor(0.3123, dtype=torch.float64, grad_fn=<MseLossBackward>)

In [55]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.0763],
        [-0.0542],
        [ 0.3221],
        [ 0.2249]], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([[-0.3051],
        [-0.1099],
        [ 0.7828]], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([[-0.0032],
        [-0.0108],
        [ 0.9855]], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([[-0.3092],
        [ 0.8493]], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([[ 0.0013],
        [ 0.9976],
        [-1.0488]], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([], size=(0, 2), dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([0.0632], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([-1.2436], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([1.3353], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([-1.0654], dtype=torch.float64, requir

In [56]:
net.weights

{5: Parameter containing:
 tensor([[-0.0763],
         [-0.0542],
         [ 0.3221],
         [ 0.2249]], dtype=torch.float64, requires_grad=True),
 4: Parameter containing:
 tensor([[-0.3051],
         [-0.1099],
         [ 0.7828]], dtype=torch.float64, requires_grad=True),
 3: Parameter containing:
 tensor([[-0.0032],
         [-0.0108],
         [ 0.9855]], dtype=torch.float64, requires_grad=True),
 1: Parameter containing:
 tensor([[-0.3092],
         [ 0.8493]], dtype=torch.float64, requires_grad=True),
 2: Parameter containing:
 tensor([[ 0.0013],
         [ 0.9976],
         [-1.0488]], dtype=torch.float64, requires_grad=True),
 0: Parameter containing:
 tensor([], size=(0, 2), dtype=torch.float64, requires_grad=True)}

In [57]:
net._outputs

{0: Parameter containing:
 tensor([0.2507, 0.3610], dtype=torch.float64, requires_grad=True),
 1: tensor([0.3023], dtype=torch.float64, grad_fn=<SigmoidBackward>),
 2: tensor([0.5806], dtype=torch.float64, grad_fn=<SigmoidBackward>),
 3: tensor([0.8702], dtype=torch.float64, grad_fn=<SigmoidBackward>),
 4: tensor([0.3366], dtype=torch.float64, grad_fn=<SigmoidBackward>),
 5: tensor([0.5492], dtype=torch.float64, grad_fn=<SigmoidBackward>)}

In [58]:
data = torch.rand(1, 1,2, dtype=torch.float64)
label = [xor(val[0][0], val[0][1]) for val in data]


In [59]:
# net(data)

In [60]:
# I.e., `output = f(cat(inputs) + bias)` for a given activation function f

def forward(layers, x):
    layer_outputs = {}
    curr_value = x
    for ii in range(len(layers)):
        print(ii)
        layer = layers[ii]
        if layer['is_output_layer']:
            # return curr_value
            return(layer_outputs)
        input_layers = [layer_outputs[layer_id] for layer_id in layer['input_layers']]
    return(layer_outputs)
        # layer_outputs[ii] = torch.relu(
        #     torch.add(
        #         torch.matmul(
        #             torch.cat(layer['input_layers']),
        #             layer['weights']

        #         ),
        #         layer['bias']
        #     )
        # )
        # curr_value = torch.sigmoid(torch.matmul(curr_value + layers[ii]['bias'], layers[ii]['out_tensor']) )

In [61]:
# forward(layers, x_input)

In [62]:
h.nodes[0].bias

-0.32540032267570496

In [63]:
with MethodTimer('Parse NNEAT'):
    myNeat = nneat(h, p.config)

The function - Parse NNEAT - has just started at 1628331620.060509
The function - Parse NNEAT - took 0.0007951259613037109 seconds to complete


In [64]:
myConnection = (-1,0)

In [65]:
h.connections[myConnection].weight

KeyError: (-1, 0)

In [102]:
for param in myNeat.parameters():
    print(param)

Parameter containing:
tensor([[-0.1703],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.1490]], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([[0.0000],
        [0.0000],
        [0.5530],
        [0.5530]], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([[0.9151, 0.0000],
        [1.0000, 1.0000]], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([[0.0701],
        [0.0000],
        [1.0000]], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([[0.5281],
        [0.0000],
        [1.0000]], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([], size=(0, 2), dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([-0.4007], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([1.2643], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([-0.8664,  2.4587], dtype=torch.float64, requires_grad=True)
Parameter conta

In [108]:
learning_rate = 0.1
# x_input = torch.randn(1, 1, 2, dtype=torch.float64)
with MethodTimer('Backprop benchmark (10k inputs)'):

    for k in range(25):
        for n in range(400):
            x_input = nn.Parameter(torch.tensor([np.random.rand(), np.random.rand()], dtype=torch.float64))


            result = myNeat.forward(x_input)
            expected_result = torch.tensor([xor(x_input[0], x_input[1])], dtype=torch.float64)
            # print(x_input)
            # print(expected_result)
            # print(result)
            loss_fn = torch.nn.MSELoss(reduction='sum')
            loss = loss_fn(result, expected_result)
            myNeat.zero_grad()
            loss.backward()
            with torch.no_grad():
                for parameter in myNeat.parameters():
                    try:
                        parameter.data -= learning_rate*parameter.grad.data
                    except AttributeError:
                        continue
                    # weight -= learning_rate * weight.grad
                # for  w_id, weight in net.weights.items():
                #     try:
                #         # print(weight.grad)
                #         weight -= learning_rate * weight.grad
                #     except TypeError:
                #         continue
                # for  b_id, bias in net.biases.items():
                #     try:
                #         bias -= learning_rate * bias.grad
                #     except TypeError:
                #         continue



The function - Backprop benchmark (10k inputs) - has just started at 1615095693.196116
The function - Backprop benchmark (10k inputs) - took 0.0015311241149902344 seconds to complete


AttributeError: 'list' object has no attribute 'size'

In [104]:
for param in myNeat.parameters():
    print(param)

Parameter containing:
tensor([[ 0.6434],
        [ 1.2305],
        [-0.4451],
        [-0.3518],
        [-0.3164]], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([[-0.8560],
        [-0.8797],
        [-0.3112],
        [-1.2250]], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([[0.8507, 0.0422],
        [0.9348, 1.0279]], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([[-0.3604],
        [-0.8499],
        [ 0.4200]], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([[-0.1479],
        [-1.3651],
        [ 0.6890]], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([], size=(0, 2), dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([-3.3236], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([-0.6114], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([-1.0203,  2.5215], dtype=torch.float64, requires_grad=True)
Para

In [91]:
myNeat.genome

In [92]:
h.connections[myConnection].weight

-0.1703117936849594

In [93]:
myNeat.genome.connections[myConnection].weight

-0.1703117936849594

In [94]:
myNeat.weights

{5: Parameter containing:
 tensor([[-0.1393],
         [ 0.2148],
         [ 0.0367],
         [-0.0018],
         [ 0.1243]], dtype=torch.float64, requires_grad=True),
 2: Parameter containing:
 tensor([[-0.1744],
         [-0.1848],
         [ 0.3618],
         [ 0.1876]], dtype=torch.float64, requires_grad=True),
 1: Parameter containing:
 tensor([[ 0.9029, -0.0036],
         [ 0.9850,  0.9993]], dtype=torch.float64, requires_grad=True),
 3: Parameter containing:
 tensor([[-0.0801],
         [-0.2856],
         [ 0.7472]], dtype=torch.float64, requires_grad=True),
 4: Parameter containing:
 tensor([[ 0.3380],
         [-0.3733],
         [ 0.8678]], dtype=torch.float64, requires_grad=True),
 0: Parameter containing:
 tensor([], size=(0, 2), dtype=torch.float64, requires_grad=True)}

In [95]:
myNeat.update_genome_weights()

In [96]:
h.connections[myConnection].weight

-0.1703117936849594

In [97]:
myNeat.genome.connections[myConnection].weight

-0.1703117936849594

In [98]:
myNeat.genome.connections[myConnection].new_weight

tensor(-0.1393, dtype=torch.float64, grad_fn=<SelectBackward>)